In [5]:
import numpy as np
import pandas as pd
import mglearn as mg
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder , StandardScaler , LabelEncoder , OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier , KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import seaborn as sb

0- reading the data

In [14]:
aisles = pd.read_csv(r"archive\aisles.csv")
department = pd.read_csv(r"archive\departments.csv")
pro_prior = pd.read_csv(r"archive\order_products__prior.csv")
pro_train = pd.read_csv(r"archive/order_products__train.csv")
orders = pd.read_csv(r"archive\orders.csv")
products = pd.read_csv(r"archive\products.csv")

In [15]:
print(pro_prior.shape)
print(orders.shape)

(32434489, 4)
(3421083, 7)


1- Join (merging the data)

In [ ]:
m1 = pd.merge(pro_prior , orders , on = "order_id" , how = "left")
m2 = pd.merge(m1 , products , on = "product_id" , how = "left")
m3 = pd.merge(m2 , department , on = "department_id" , how = "left")
Full_DataSet = pd.merge(m3 , aisles , on = "aisle_id" , how = "left")

In [26]:
#اخترنا هذول لانهم يعتبرو العامود الفقري للداتا سيت , لانهم بحتوي على الشغلات الرئيسيه مثل نوع الطلب واليوز وكذا
check = pro_prior.merge(orders , on = "order_id" , how = "outer" , indicator = True)
check["_merge"].value_counts()

_merge
both          32434489
right_only      206209
left_only            0
Name: count, dtype: int64

In [27]:
#هون اعملو ميموري , بعدين اعملو الداتا سيت الكامله بملف منفصل 
#Full_DataSet.to_csv('archive/full_instacart_data.csv', index=False)
#كنسلنا عيلها لانه طلع معنا مساحتها 3 قيقا :)